# Oil and Gas Production and Emissions Data on the Norwegian Continental Shelf

## Part 4

This is the fourth part of a series of notebooks that I am creating to analyze the oil and gas production and emissions data on the Norwegian Continental Shelf. The data is provided by the Norwegian Petroleum Directorate (NPD) and covers the period from 2001 to 2020. The data is available on the NPD website and can be downloaded from the following link: [Production and Emissions Data](https://factpages.sodir.no/).

You can find the other parts of the series here:

#### Part 1: [Data Collection](https://github.com/percw/Norwegian_oil_gas_decarbonization/blob/main/notebooks/01_data_building/01_production_and_emission_data_building.ipynb)

#### Part 2: [Data Cleaning](https://github.com/percw/Norwegian_oil_gas_decarbonization/blob/main/notebooks/02_data_cleaning/02_production_and_emission_data_cleaning.ipynb)

#### Part 3 [Data Processing](https://github.com/percw/Norwegian_oil_gas_decarbonization/blob/main/notebooks/03_data_processing/03_production_and_emission_data_processing.ipynb)


# Table of contents

1. [Imports](#imports)
2. [Causal Discovery](#causal_discovery)
3. [Linear Regression](#linear_regression)
4. [Double Machine Learning](#Double-Machine-Learning)


## Imports


In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

pd.set_option("display.max_columns", None)

In [16]:
# Importing the dataset from the csv file
filepath = "https://raw.githubusercontent.com/percw/Norwegian_oil_gas_decarbonization/main/data/output/emissions_and_production/cleaned/fields_prod_emissions_intensities_share_1997_2023.csv"

# Creating a check if import is successful
try:
    data = pd.read_csv(filepath, sep=",")
    print("Data import successful")
except:
    print("Data import failed")

Data import successful


## Electrification Effect Using Difference-in-Difference Analysis

We want to check how electrification of the oil and gas platforms in the Norwegian Continental Shelf has affected emissions. We will use the diff-in-diff method to estimate the effect of electrification on emissions.

Diff-in-diff is a method used to estimate the causal effect of a treatment on a group of units in an observational study. The method compares the average change in the outcome variable for the treated group to the average change in the outcome variable for the control group. The difference between these two changes is the estimated causal effect of the treatment.


In [30]:
data.head(3)

,field,year,net_oil_prod_yearly_mill_sm3,net_gas_prod_yearly_bill_sm3,net_ngl_prod_yearly_mill_sm3,net_condensate_prod_yearly_mill_sm3,net_oil_eq_prod_yearly_mill_sm3,produced_water_yearly_mill_sm3,field_id,net_oil_prod_monthly_sm3_volatility,net_gas_prod_monthly_sm3_volatility,net_ngl_prod_monthly_sm3_volatility,net_condensate_prod_monthly_sm3_volatility,net_oil_eq_prod_monthly_sm3_volatility,produced_water_in_field_volatility,status,current_status,field_owner,processing_field,field_in_emissions,facilities_lifetime_mean,facilities_lifetime_std,facilities_water_depth_mean,facilities_water_depth_std,subsea_facilites_shut_down,surface_facilites_shut_down,subsea_facilites_in_service,surface_facilites_in_service,facility_kind_multi well template,facility_kind_single well template,facility_kind_offshore wind turbine,facility_kind_subsea structure,facility_kind_fpso,facility_kind_jacket 8 legs,facility_kind_condeep monoshaft,facility_kind_loading system,facility_kind_jacket 4 legs,facility_kind_jacket tripod,facility_kind_fsu,facility_kind_semisub steel,facility_kind_condeep 4 shafts,facility_kind_landfall,facility_kind_tlp concrete,facility_kind_jack-up 3 legs,facility_kind_jacket 6 legs,facility_kind_tlp steel,facility_kind_semisub concrete,facility_kind_mopustor,facility_kind_spar,well_status_closed,well_status_drilling,well_status_injecting,well_status_junked,well_status_online/operational,well_status_p&a,well_status_plugged,well_status_producing,well_status_suspended,well_purpose_injection,well_purpose_observation,well_purpose_production,well_subsea_no,well_subsea_yes,well_final_vertical_depth_mean,well_final_vertical_depth_std,well_water_depth_mean,well_water_depth_std,investments_mill_nok,future_investments_mill_nok,yearly_co2_emissions_1000_tonnes,org_number,operator,yearly_ch4_emissions_tons,yearly_nox_emissions_tons,yearly_oil_spill_emissions_tons,yearly_water_emissions_m3,ownership_original,ownership_new_name,current_remaining_recoverable_oil,current_remaining_recoverable_gas,current_remaining_recoverable_ngl,current_remaining_recoverable_condensate,current_remaining_recoverable_oe,original_recoverable_oil,original_recoverable_gas,original_recoverable_ngl,original_recoverable_condensate,original_recoverable_oe,electrified,years_electrified,electricity_mw,imported_power_2023_gwh/y,yearly_tco2e_gwp100,yearly_tco2e_gwp20,emission_intensity,share_peak_prod,oe_cum_sum_prod,share_reserve_of_original_reserve,oe_fac_prod_mill_sm3,yearly_fac_tco2e_gwp100,oe_share_prod,yearly_tco2e_prod_share_emissions,emission_intensity_share,gas_reserve_ratio,oil_reserve_ratio,oil_gas_reserve_ratio,treatment,post,treatment_post,predicted
0,statfjord nord,1997-01-01,3.93531,0.17288,0.09230,0.00000,4.20051,0.0,43679,0.053898,0.000405,0.000334,0.000000,0.054005,0.0,Producing,Producing,21084.0,statfjord,True,27.5,15.0,241.75,65.030121,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,9.0,0.0,4.0,0.0,13.0,0.0,17.0,2888.8,0.000000,285.0,0.0,255.0,2946.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Den norske stats oljeselskap a.s': 50.0, 'Mo...","{'Equinor ASA': 1.875, 'Mobil Development Norw...",3.72,0.23,0.05,0.0,4.05,44.12,2.37,1.12,0.00,48.62,0,0,0,0.0,NaN,NaN,0.000000,100.0000,4.20051,91.360531,32.43504,3.153630e+06,0.129505,408411.794667,115.748925,0.000049,0.907445,18.616034,0,False,0,-157.624060
1,veslefrikk,1997-01-01,3.47468,0.13919,0.08596,0.00000,3.69981,0.0,43618,0.052851,0.008952,0.005461,0.000000,0.053418,0.0,Producing,Shut down,21212.0,veslefrikk,True,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111.0,0.0,0.0,69.0,6.0,36.0,111.0,0.0,3317.0,0.000000,175.0,0.0,229.0,0.0,156.20492,993246905.0,equinor energy as,64.319881,973.4661,88.599389,2782760.0,"{'Den norske stats oljeselskap a.s': 55.0, 'To...","{'Equinor ASA': 55.0, 'TotalEnergies EP Norge'...",0.00,0.00,0.00,0.0,0.00,55.34,4.1

In [32]:
data["yearly_tco2_emissions"] = data["yearly_co2_emissions_1000_tonnes"] / 1000

### Emissions


In [33]:
# Convert the 'year' column to datetime format if it's not already
data["year"] = pd.to_datetime(data["year"], format="%Y")

# Create a treatment variable
# 'electrified' is the binary treatment variable and it becomes 1 when electrified
data["treatment"] = data["electrified"]

# Create a post variable
# 'years_electrified' is the number of years since electrification, a field is considered post-treatment when years_electrified > 0
data["post"] = data["years_electrified"] > 0

# Create an interaction term for DiD
data["treatment_post"] = data["treatment"] * data["post"]

# Select the dependent variable
dependent_var = "yearly_tco2_emissions"
dependent_var_share = "yearly_tco2e_prod_share_emissions"

# Specify the model
# We are looking at the effect of the treatment, post, and their interaction (DiD)
formula = f"{dependent_var} ~ treatment + post + treatment_post + C(year) + C(field_id)"
formula_share = (
    f"{dependent_var_share} ~ treatment + post + treatment_post + C(year) + C(field_id)"
)

# Fit the model
model = ols(formula, data=data).fit()
model_share = ols(formula_share, data=data).fit()

# Display the results
print(model.summary())
print(model_share.summary())

                              OLS Regression Results                             
Dep. Variable:     yearly_tco2_emissions   R-squared:                       0.920
Model:                               OLS   Adj. R-squared:                  0.911
Method:                    Least Squares   F-statistic:                     106.3
Date:                   Mon, 15 Jul 2024   Prob (F-statistic):               0.00
Time:                           13:27:01   Log-Likelihood:                 1168.4
No. Observations:                   1093   AIC:                            -2123.
Df Residuals:                        986   BIC:                            -1588.
Df Model:                            106                                         
Covariance Type:               nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

### Emission intensity


In [23]:
# Rename "kgco2e/toe_int_gwp100" to "emission_intensity"
# rename "share_intensity_tco2e/toe_gwp100" to "emission_intensity_share"

data = data.rename(
    columns={
        "share_intensity_tco2e/toe_gwp100": "emission_intensity_share",
        "kgco2e/toe_int_gwp100": "emission_intensity",
    }
)

In [25]:
# Convert the 'year' column to datetime format if it's not already
data["year"] = pd.to_datetime(data["year"], format="%Y")

# Create a treatment variable
# 'electrified' is the binary treatment variable and it becomes 1 when electrified
data["treatment"] = data["electrified"]

# Create a post variable
# 'years_electrified' is the number of years since electrification, a field is considered post-treatment when years_electrified > 0
data["post"] = data["years_electrified"] > 0

# Create an interaction term for DiD
data["treatment_post"] = data["treatment"] * data["post"]

# Select the dependent variable
dependent_var = "emission_intensity"
dependent_var_share = "emission_intensity_share"


# Specify the model
# We are looking at the effect of the treatment, post, and their interaction (DiD)
formula = f"{dependent_var} ~ treatment + post + treatment_post + C(year) + C(field_id)"
formula_share = (
    f"{dependent_var_share} ~ treatment + post + treatment_post + C(year) + C(field_id)"
)

# Fit the model
model = ols(formula, data=data).fit()
model_share = ols(formula_share, data=data).fit()

# Display the results
print(model.summary())
print(model_share.summary())

                            OLS Regression Results                            
Dep. Variable:     emission_intensity   R-squared:                       0.438
Model:                            OLS   Adj. R-squared:                  0.381
Method:                 Least Squares   F-statistic:                     7.739
Date:                Mon, 15 Jul 2024   Prob (F-statistic):           1.75e-94
Time:                        13:07:34   Log-Likelihood:                -10318.
No. Observations:                1445   AIC:                         2.090e+04
Df Residuals:                    1312   BIC:                         2.160e+04
Df Model:                         132                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

### Overview from Difference-in-Differences Analysis

The Difference-in-Differences (DiD) analysis evaluates the impact of electrification on multiple emission variables for Norwegian oil fields from 1997 to 2022. The key metrics assessed are:

1. **CO2e Emissions**
2. **Production Share CO2e Emissions**
3. **Emission Intensity**
4. **Production Share Emission Intensity**

#### Key Findings

| Metric                         | CO2e Emissions | Production Share CO2e Emissions | Emission Intensity | Production Share Emission Intensity |
| ------------------------------ | -------------- | ------------------------------- | ------------------ | ----------------------------------- |
| **R-squared**                  | 0.920          | 0.837                           | 0.438              | 0.612                               |
| **Adj. R-squared**             | 0.911          | 0.820                           | 0.381              | 0.573                               |
| **Treatment Coefficient**      | -164.256       | -211800                         | 183.484            | -47.071                             |
| **Treatment p-value**          | 0.011          | 0.184                           | 0.435              | 0.587                               |
| **Interaction Coefficient**    | 58.725         | 251.900                         | 391.658            | -9.140                              |
| **Interaction p-value**        | 0.389          | 0.132                           | 0.112              | 0.920                               |
| **Post-Treatment Coefficient** | 76.420         | 429.900                         | -549.120           | -33.292                             |
| **Post-Treatment p-value**     | 0.000          | 0.005                           | 0.016              | 0.690                               |


## Regression


In this notebook, I will use the data that I have processed in the previous notebooks to perform an Ordinary Least Squares (OLS) regression analysis. The goal is to understand the relationship between the oil and gas production and the CO2 emissions on the Norwegian Continental Shelf. I will also analyze the relationship between the oil and gas production and the methane emissions.

The degree of complexity will increase as the series progresses. I will start with a simple model and then add more variables to the regression analysis. I will also analyze the residuals to check the assumptions of the OLS regression.


### Static Variables Regression

#### The $\beta$'s to explore (independent variables):

**Starting point:**

1. Share of production of peak production
2. Gas/Oil Ratio
3. Years in production
4. Water depth
5. Original reserve
6. Remaining reserve
7. _Control: Time_
8. _Fixed effects_

**Advancements:**

1.
1. Carbon Price (inflation and currency adjusted)
1. Oil price (inflation and currency adjusted) 9.


| Variable                                    | Description/Unit                       | Name in the dataframe                  |
| ------------------------------------------- | -------------------------------------- | -------------------------------------- |
| **Y1 : Distributed emission intensity**     | tCO2e/toe (dep. variable 1)            | share_intensity_tco2e/toe_gwp100       |
| **Y2 : Emission intensity**                 | tCO2e/toe (dep. variable 2)            | kgco2e/toe_int_gwp100                  |
| Share of production of peak production      | Share of production of peak production | share_peak_prod                        |
| Oil/gas Ratio                               | Oil/gas Ratio                          | gas_oil_ratio                          |
| Well water depth                            | Water depth                            | well_water_depth_mean                  |
| Well depth                                  | Water depth                            | well_final_vertical_depth_mean         |
| Original reserve                            | Original reserve                       | original_recoverable_oe                |
| Share remaining reserve of original reserve | %                                      | share_reserve_of_original_reserve      |
| Oil eq. production volatility               | Volatility                             | net_oil_eq_prod_monthly_sm3_volatility |


In [ ]:
# Show all columns

data.head()
data.columns.tolist()

['field',
 'year',
 'net_oil_prod_yearly_mill_sm3',
 'net_gas_prod_yearly_bill_sm3',
 'net_ngl_prod_yearly_mill_sm3',
 'net_condensate_prod_yearly_mill_sm3',
 'net_oil_eq_prod_yearly_mill_sm3',
 'produced_water_yearly_mill_sm3',
 'field_id',
 'net_oil_prod_monthly_sm3_volatility',
 'net_gas_prod_monthly_sm3_volatility',
 'net_ngl_prod_monthly_sm3_volatility',
 'net_condensate_prod_monthly_sm3_volatility',
 'net_oil_eq_prod_monthly_sm3_volatility',
 'produced_water_in_field_volatility',
 'status',
 'current_status',
 'field_owner',
 'processing_field',
 'field_in_emissions',
 'facilities_lifetime_mean',
 'facilities_lifetime_std',
 'facilities_water_depth_mean',
 'facilities_water_depth_std',
 'subsea_facilites_shut_down',
 'surface_facilites_shut_down',
 'subsea_facilites_in_service',
 'surface_facilites_in_service',
 'facility_kind_multi well template',
 'facility_kind_single well template',
 'facility_kind_offshore wind turbine',
 'facility_kind_subsea structure',
 'facility_kind_fps

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# Preprocess the Data: Remove np.nan and inf values
ols_data = data.replace([np.inf, -np.inf], np.nan).dropna()

# Define the dependent and independent variables
Y1 = ols_data["share_intensity_tco2e/toe_gwp100"]
Y2 = ols_data["kgco2e/toe_int_gwp100"]

X = ols_data[
    [
        "share_peak_prod",
        "well_water_depth_mean",
        "well_final_vertical_depth_mean",
        "original_recoverable_oe",
        "share_reserve_of_original_reserve",
        "net_oil_eq_prod_monthly_sm3_volatility",
        "gas_reserve_ratio",
        "oil_gas_reserve_ratio",
    ]
]

# Perform OLS regression using statsmodels for detailed results
ols_Y1 = sm.OLS(Y1, X).fit()
ols_Y2 = sm.OLS(Y2, X).fit()

# Conduct the OLS using sklearn for consistency with provided code
reg = LinearRegression().fit(X, Y2)

# Second degree polynomial regression
X_poly2 = X.copy()
for col in X.columns:
    if col != "const":
        X_poly2[f"{col}^2"] = X[col] ** 2

ols_Y1_poly2 = sm.OLS(Y1, X_poly2).fit()
ols_Y2_poly2 = sm.OLS(Y2, X_poly2).fit()


# Define a function to display the results in a pretty format
def pretty_print_results(results, degree):
    print(f"OLS Regression Results (Degree {degree}):")
    print("========================================")
    print(results.summary())
    print("\n\n")


# Print the results
pretty_print_results(ols_Y1, 1)
pretty_print_results(ols_Y2, 1)
pretty_print_results(ols_Y1_poly2, 2)
pretty_print_results(ols_Y2_poly2, 2)

OLS Regression Results (Degree 1):
                                   OLS Regression Results                                   
Dep. Variable:     share_intensity_tco2e/toe_gwp100   R-squared:                       0.282
Model:                                          OLS   Adj. R-squared:                  0.273
Method:                               Least Squares   F-statistic:                     35.12
Date:                              Wed, 26 Jun 2024   Prob (F-statistic):           5.97e-47
Time:                                      00:06:16   Log-Likelihood:                -4655.7
No. Observations:                               726   AIC:                             9329.
Df Residuals:                                   717   BIC:                             9371.
Df Model:                                         8                                         
Covariance Type:                          nonrobust                                         
                                   

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from statsmodels.tools.sm_exceptions import MissingDataError

data = pd.read_csv(filepath, sep=",")

# Ensure the DataFrame has a MultiIndex for panel data
data.set_index(["field_id", "year"], inplace=True)

# Define the dependent variable
Y1 = data["share_intensity_tco2e/toe_gwp100"]

# Define the independent variables
X = data[
    [
        "share_peak_prod",
        "well_water_depth_mean",
        "well_final_vertical_depth_mean",
        "original_recoverable_oe",
        "share_reserve_of_original_reserve",
        "net_oil_eq_prod_monthly_sm3_volatility",
        "gas_reserve_ratio",
        # "oil_gas_reserve_ratio",
    ]
]

# Add a constant term for the intercept
X = sm.add_constant(X)

# Ensure Y1 aligns with X after dropping NaNs
X = X.dropna()
Y1 = Y1.loc[X.index]

# Perform OLS regression using statsmodels for detailed results
try:
    ols_Y1 = sm.OLS(Y1, X).fit()
except MissingDataError:
    print("Missing data found in the independent variables")

# Fixed Effects Model
try:
    X_fe = X.drop(columns=["const"])
    fixed_effects_Y1 = PanelOLS(Y1, X_fe, time_effects=True).fit()
except MissingDataError:
    print("Missing data found in the fixed effects model")


# Define a function to display the results in a pretty format
def pretty_print_results(results, degree):
    print(f"OLS Regression Results (Degree {degree}):")
    print("========================================")
    print(results.summary())
    print("\n\n")


def pretty_print_fe_results(results):
    print("Fixed Effects Regression Results:")
    print("=================================")
    print(results)
    print("\n\n")


# Print the results
pretty_print_results(ols_Y1, 1)
pretty_print_fe_results(fixed_effects_Y1)

OLS Regression Results (Degree 1):
                                   OLS Regression Results                                   
Dep. Variable:     share_intensity_tco2e/toe_gwp100   R-squared:                       0.226
Model:                                          OLS   Adj. R-squared:                  0.222
Method:                               Least Squares   F-statistic:                     59.90
Date:                              Wed, 26 Jun 2024   Prob (F-statistic):           1.34e-75
Time:                                      09:06:42   Log-Likelihood:                -9373.1
No. Observations:                              1445   AIC:                         1.876e+04
Df Residuals:                                  1437   BIC:                         1.880e+04
Df Model:                                         7                                         
Covariance Type:                          nonrobust                                         
                                   

### Dynamic (operational) Variables Regression

#### The $\beta$'s to explore (independent variables):

1. Operator
2. Owner
3. Electrified
4. Production Volatility
5. Well Status : Drilling
6. Well Status : Open/Producing
7. Well Status : Plugged
8. Well Status : Closed
9. Well Final Vertical Depth Mean
10.
11.
12.

++ Controls


In [ ]:
data.head(5)

## Double Machine Learning


### Introduction to Double Machine Learning (DML) Regression

The Double Machine Learning (DML) regression framework aims to accurately estimate the causal effects of production levels and the quantity left in the field on emission levels, while effectively accounting for other confounding factors that could influence these relationships. The model decomposes the relationship into two primary stages. In the first stage, machine learning algorithms are utilized to predict the treatment variables (production level and quantity left in the field) based on a set of control variables. This step helps isolate the variation in the treatment variables that is not explained by the control variables. By doing so, it creates residuals—essentially, the unexplained parts of the treatment variables that are free from the confounding influence of the controls. In the second stage, these residuals are used in a regression model to predict the outcome variable (emissions), along with their interaction terms and squared terms to capture potential nonlinear effects and interactions. This approach ensures that the estimated effects of the treatment variables on emissions are unbiased and more reliable, as it rigorously controls for the confounding influence of other variables. The ultimate goal is to provide a clearer understanding of how changes in production and field practices directly impact emissions, free from the distortions caused by other factors.


### Double Machine Learning (DML) Regression of Carbon Intensity

Double Machine Learning (DML) represents a cutting-edge econometric framework tailored for robust causal inference amidst high-dimensional data and intricate confounding structures. DML employs sophisticated machine learning techniques to meticulously control for confounders, enabling the precise isolation of causal effects of treatment variables on an outcome. This methodology is particularly advantageous in scenarios involving complex datasets where conventional regression approaches may inadequately address the multiplicity of confounding factors.

### Model Specification

Consider the following DML regression model where the outcome variable $ Y $ denotes emission levels. The treatment variables are $ X_1 $ (production level) and $ X_2 $ (quantity left in the field), while $ Z $ represents additional control variables. Interaction terms between production and the quantity left in the field are included to capture potential synergistic effects.

$
Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_2^2 + \beta_4 (X_1 \cdot X_2) + \beta_5 (X_1 \cdot X_2^2) + G(Z) + u
$

### Components of the Model

- **Outcome Variable $ Y $**: Represents the emission levels.
- **Treatment Variables $ X_1 $ and $ X_2 $**:
  - $ X_1 $: Production level.
  - $ X_2 $: Quantity left in the field.
- **Interaction Terms**:
  - $ X_1 \cdot X_2 $: Interaction between production and quantity left in the field.
  - $ X_1 \cdot X_2^2 $: Interaction between production and the squared quantity left in the field.
- **Control Function $ G(Z) $**: A non-parametric function representing the influence of other covariates $ Z $ on $ Y $.
- **Error Term $ u $**: Captures unobserved factors affecting $ Y $.

### Estimation Process Using DML

1. **First Stage**:

   1. **Modeling $ X_1 $ and $ X_2 $**: Use machine learning methods (e.g., Lasso, Random Forest, Neural Networks) to predict the treatment variables $ X_1 $ and $ X_2 $ based on $ Z $. This helps control for the confounding effect of $ Z $.
   2. **Residuals**: Calculate the residuals of $ X_1 $ and $ X_2 $ after accounting for $ Z $.

2. **Second Stage**:
   1. **Predicting $ Y $**: Use the residuals obtained from the first stage to predict $ Y $, including the interaction terms and their non-linear components.
   2. **Control Function**: Include $ G(Z) $ to control for the influence of $ Z $ on $ Y $.

### Mathematical Formulation

The DML estimation process involves the following steps:

1. **Estimate the nuisance parameters**: $ \hat{m}\_1(Z) $ and $ \hat{m}\_2(Z) $ for $ X_1 $ and $ X_2 $ respectively.
2. **Compute residuals**:
   $
   \hat{u}_1 = X_1 - \hat{m}_1(Z), \quad \hat{u}_2 = X_2 - \hat{m}_2(Z)
   $
3. **Estimate the reduced form**:
   $
   Y = \beta_0 + \beta_1 \hat{u}_1 + \beta_2 \hat{u}_2 + \beta_3 \hat{u}_2^2 + \beta_4 (\hat{u}_1 \cdot \hat{u}_2) + \beta_5 (\hat{u}_1 \cdot \hat{u}_2^2) + G(Z) + u
   $
